In [14]:
from qiskit_nature.drivers import Molecule
molecule = Molecule(geometry=[['H', [0., 0., 0.]],
                              ['H', [0., 0., 0.735]]],
                     charge=0, multiplicity=1)

In [15]:
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
driver = ElectronicStructureMoleculeDriver(molecule, basis='sto3g', driver_type=ElectronicStructureDriverType.PYSCF)
_ = driver.run()

In [22]:
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
hamiltonian = ElectronicStructureProblem(driver)
operators_secq = hamiltonian.second_q_ops()

In [34]:
particle_number = hamiltonian.grouped_property_transformed.get_property("ParticleNumber")
print(f"Hydrogen: {particle_number.num_alpha} alpha (spin up) electron, "
      f"{particle_number.num_beta} beta (spin down) electron, {particle_number.num_spin_orbitals} spin orbitals")

Hydrogen: 1 alpha (spin up) electron, 1 beta (spin down) electron, 4 spin orbitals


In [20]:
electronic_energy = hamiltonian.grouped_property_transformed.get_property("ElectronicEnergy")
print(electronic_energy)

ElectronicEnergy
	(AO) 1-Body Terms:
		Alpha
		<(2, 2) matrix with 4 non-zero entries>
		[0, 0] = -1.1242175791954514
		[0, 1] = -0.9652573993472754
		[1, 0] = -0.9652573993472754
		[1, 1] = -1.1242175791954514
		Beta
		Same values as the corresponding primary-spin data.
	(AO) 2-Body Terms:
		Alpha-Alpha
		<(2, 2, 2, 2) matrix with 16 non-zero entries>
		[0, 0, 0, 0] = 0.7746059439198978
		[0, 0, 0, 1] = 0.4474457245330949
		[0, 0, 1, 0] = 0.447445724533095
		[0, 0, 1, 1] = 0.5718769760004512
		[0, 1, 0, 0] = 0.4474457245330951
		... skipping 11 entries
		Beta-Alpha
		Same values as the corresponding primary-spin data.
		Beta-Beta
		Same values as the corresponding primary-spin data.
		Alpha-Beta
		Same values as the corresponding primary-spin data.
	(MO) 1-Body Terms:
		Alpha
		<(2, 2) matrix with 2 non-zero entries>
		[0, 0] = -1.2563390730032498
		[1, 1] = -0.47189600728114245
		Beta
		Same values as the corresponding primary-spin data.
	(MO) 2-Body Terms:
		Alpha-Alpha
		<(2, 2, 2,

In [30]:
electronic_energy_operator = operators_secq["ElectronicEnergy"]
electronic_energy_operator.set_truncation(0)
print(electronic_energy_operator.to_matrix())

  (1, 1)	(-0.47189600728114245+0j)
  (2, 2)	(-1.2563390730032498+0j)
  (3, 3)	(-1.244584549813327+0j)
  (4, 4)	(-0.47189600728114245+0j)
  (5, 5)	(-0.24521829183026678+0j)
  (10, 5)	(0.1809311997842311+0j)
  (6, 6)	(-1.063653350029096+0j)
  (9, 6)	(0.18093119978423128-0j)
  (7, 7)	(-0.3533251041071551+0j)
  (8, 8)	(-1.2563390730032498+0j)
  (6, 9)	(0.18093119978423128-0j)
  (9, 9)	(-1.063653350029096+0j)
  (5, 10)	(0.18093119978423147+0j)
  (10, 10)	(-1.8369679912029833+0j)
  (11, 11)	(-1.1606317377577642+0j)
  (12, 12)	(-1.244584549813327+0j)
  (13, 13)	(-0.353325104107155+0j)
  (14, 14)	(-1.1606317377577642+0j)
  (15, 15)	(0.21427823841947302+0j)


In [37]:
# from qiskit_nature.converters.second_quantization import QubitConverter
# from qiskit_nature.operators.second_quantization import FermionicOp
# from qiskit_nature.mappers.second_quantization import JordanWignerMapper
# converter = JordanWignerMapper()
# qubit_converter = QubitConverter(converter)
# for label in ["+_0", "+_1", "+_2", "+_3", "+_4"]:
#     fermionic_operator = FermionicOp(label)
#     qubit_operator = qubit_converter.convert(fermionic_operator)
#     print(f"{label}: {qubit_operator}")

+_0: 0.5 * X
+ -0.5j * Y
+_1: 0.5 * XZ
+ -0.5j * YZ
+_2: 0.5 * XZZ
+ -0.5j * YZZ
+_3: 0.5 * XZZZ
+ -0.5j * YZZZ
+_4: 0.5 * XZZZZ
+ -0.5j * YZZZZ


In [48]:
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
converter = JordanWignerMapper()  # also ParityMapper, BravyiKitaevMapper
qubit_converter = QubitConverter(converter, two_qubit_reduction=True)
qubit_operator = qubit_converter.convert(electronic_energy_operator)
print(qubit_operator)

-0.8105479805373279 * IIII
+ 0.1721839326191554 * IIIZ
- 0.22575349222402372 * IIZI
+ 0.17218393261915543 * IZII
- 0.2257534922240237 * ZIII
+ 0.12091263261776627 * IIZZ
+ 0.16892753870087907 * IZIZ
+ 0.045232799946057826 * YYYY
+ 0.045232799946057826 * XXYY
+ 0.045232799946057826 * YYXX
+ 0.045232799946057826 * XXXX
+ 0.1661454325638241 * ZIIZ
+ 0.1661454325638241 * IZZI
+ 0.17464343068300453 * ZIZI
+ 0.12091263261776627 * ZZII


In [47]:
from pyscf import gto
atom_PySCF = [" ".join([str(val) for val in [name, *coordinates]]) for name, coordinates in molecule.geometry]
molecule_PySCF = gto.M(atom=atom_PySCF)

['H 0.0 0.0 0.0', 'H 0.0 0.0 0.735']

[['H', [0.0, 0.0, 0.0]], ['H', [0.0, 0.0, 0.735]]]

['H 0.0 0.0 0.0', 'H 0.0 0.0 0.735']